In [2]:
import geopandas
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [3]:
perimeters = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    index='id'
)

regions = perimeters[
    perimeters.index.isin([
        'zurich', 'ebc_zrh_v01_ex_zurich'
    ])
]

In [4]:
def calculate_descriptives_for_one_scenario(accessibility, regions):
    
    # assign to regions
    accessibility = gpd.sjoin(
        accessibility,
        regions,
        how='left',
        predicate='within'
    ).rename(columns={'index_right': 'region'})
    
    accessibility_by_region = accessibility.groupby('region').agg({
        'accessibility': ['mean', 'median', 'std']
    })
    
    # add age groups
    accessibility['age_group'] = accessibility.apply(
        lambda row: '<=25' if row['age'] <= 25 else '>=65' if row['age'] >= 65 else 'other',
        axis=1
    )
    
    accessibility_by_age_group = accessibility.groupby('age_group').agg({
        'accessibility': ['mean', 'median', 'std']
    })
    
    return pd.concat([
        accessibility_by_region,
        accessibility_by_age_group
    ])



In [5]:
accessibility_before = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'accessibility.gpkg')
)

accessibility_after = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'accessibility.gpkg')
)

results = pd.concat([
    calculate_descriptives_for_one_scenario(accessibility_before, regions).add_prefix('before_'),
    calculate_descriptives_for_one_scenario(accessibility_after, regions).add_prefix('after_')
    ],
    axis=1
)

results

before_accessibility                            \
                               before_mean before_median  before_std   
ebc_zrh_v01_ex_zurich          2123.469834   2150.694833  390.297351   
zurich                         2932.164499   2938.787765  309.287204   
<=25                           2249.503543   2174.529646  531.612767   
>=65                           2410.462251   2385.140280  456.719542   
other                          2388.439180   2336.102013  525.058409   

                      after_accessibility                           
                               after_mean after_median   after_std  
ebc_zrh_v01_ex_zurich         2123.469834  2150.694833  390.297351  
zurich                        2932.164499  2938.787765  309.287204  
<=25                          2249.503543  2174.529646  531.612767  
>=65                          2410.462251  2385.140280  456.719542  
other                         2388.439180  2336.102013  525.058409